# Create ping files based on lab choices

## Lab settings

In [1]:

# This dictionnary should be the same as the one used for creating file_defs.
# Here , we quoted only those entries useful for creating ping files
settings={
    #'mips' : {'AerChemMIP','C4MIP','CFMIP','DAMIP', 'FAFMIP' , 'GeoMIP','GMMIP','ISMIP6',\
    #                  'LS3MIP','LUMIP','OMIP','PMIP','RFMIP','ScenarioMIP','CORDEX','SIMIP'},
    # If you want to get comprehensive ping files; use :
    'mips' : {"CMIP6", "AerChemMIP", "C4MIP", "CFMIP", "DAMIP", "DCPP", "FAFMIP", "GeoMIP", "GMMIP", 
              "HighResMIP", "ISMIP6", "LS3MIP", "LUMIP", "OMIP", "PDRMIP", "PMIP", "RFMIP", "ScenarioMIP", 
              "SolarMIP", "VolMIP", "CORDEX", "DynVar", "SIMIP", "VIACSAB", "SPECS", "CCMI", "CMIP5", 
              "CMIP", "DECK"},
    'max_priority' : 3,
    'tierMax'      : 3,
   # Each XIOS  context does adress a number of realms
    'realms_per_context' : { 'nemo': ['seaIce', 'ocean', 'ocean seaIce', 'ocnBgchem', 'seaIce ocean'] ,
                          'arpsfx' : ['atmos', 'atmos atmosChem', 'aerosol', 'atmos land', 'land',
                                     'landIce land',  'aerosol land','land landIce',  'landIce', ],
                          }, 
    "ping_variables_prefix" : "CMIP6_",
    # We account for a file listing the variables which the lab does not want to produce 
    # Format : MIP varname as first column, comment lines begin with '#'
    "excluded_vars_file":"/cnrm/est/USERS/senesi/public/CMIP6/data_request/cnrm/excluded_vars.txt",
    "excluded_vars" : None,
    }

### Read excluded variables list - you may skip that

In [2]:
l=[]
if settings["excluded_vars"] is None and settings["excluded_vars_file"] is not None :
    with open(settings["excluded_vars_file"],'r') as fv : varlines=fv.readlines()
    for line in varlines: 
        fields=line.split()
        if len(fields) > 0 : 
            first=fields[0]
            if first[0] != '#' :  l.append(first)
settings["excluded_vars"]=l
                       

## For getting a comprehensive ping file, one can (re)set the excluded_var list to an empty list

In [3]:
settings["excluded_vars"]=[]

In [4]:
from dr2xml import select_CMORvars_for_lab, pingFileForRealmsList

01.00.01


## Select all variables to consider, based on lab settings

In [5]:
svars=select_CMORvars_for_lab(settings, printout=True)


Number of Request Links which apply to MIPS set(['CORDEX', 'GMMIP', 'RFMIP', 'VolMIP', 'CMIP6', 'ScenarioMIP', 'GeoMIP', 'C4MIP', 'PDRMIP', 'CMIP', 'DECK', 'LUMIP', 'CMIP5', 'CFMIP', 'OMIP', 'DAMIP', 'CCMI', 'SolarMIP', 'VIACSAB', 'SIMIP', 'DCPP', 'ISMIP6', 'AerChemMIP', 'PMIP', 'FAFMIP', 'DynVar', 'LS3MIP', 'SPECS', 'HighResMIP'])  is:  355

Number of CMOR variables for these requestLinks is : 1970

Number of CMOR variables once filtered by excluded vars is : 1970

Number of variables with distinct labels is : 1302

Number of simplified vars is : 1970


## Doc for ping files create function

In [6]:
help(pingFileForRealmsList)

Help on function pingFileForRealmsList in module dr2xml:

pingFileForRealmsList(context, lrealms, svars, dummy='field_atm', dummy_with_shape=False, exact=False, comments=False, prefix='CV_', filename=None)
    Based on a list of realms LREALMS and a list of simplified vars SVARS, create the ping 
    file which name is ~ ping_<realms_list>.xml, which defines fields for all vars in SVARS, 
    with a field_ref which is either 'dummy' or '?<varname>' (depending on logical DUMMY)
    
    If EXACT is True, the match between variable realm string and one of the realm string 
    in the list must be exact. Otherwise, the variable realm must be included in (or include) 
    of the realm list strings
    
    COMMENTS, if not False nor "", will drive the writing of variable description and units 
    as an xml comment. If it is a string, it will be printed before this comment string (and  
    this allows for a line break)
    
    DUMMY, if not false, should be either 'True', for a standard 

### When using function create_ping_files with argument exact=False, each ping file will adress all variables which realm includes or is included in one of the strings in a realms set  <br><br> e.g for set ['ocean','seaIce'], ping file 'ping_ocean_seaIce.xml' will includes variables which realm is either 'ocean' or 'seaIce' or 'ocean seaIce'

## Create various ping files for various sets of realms 

In [7]:
# For e.g. CNRM-CM, a single Xios context handle this list of realms :
context='arpsfx'
realms=settings['realms_per_context'][context]
pingFileForRealmsList(context,realms,svars,comments=" ",exact=False,dummy=True, 
                      prefix=settings['ping_variables_prefix'],
                      filename='output_sample/ping_arpsfx.xml',dummy_with_shape=True)
#pingFileForRealmsList(realms,svars,comments="",exact=False,dummy=True,filename='ping_atmos_nocomments.xml')


818 variables written for output_sample/ping_arpsfx.xml


In [8]:
# For Nemo, the context handle that realms
if True : pingFileForRealmsList('nemo',['ocean','seaIce','ocnBgchem'],svars,comments=" ",exact=False,dummy=True,
                      prefix=settings['ping_variables_prefix'],
                      filename='output_sample/ping_nemo.xml',dummy_with_shape=True)

485 variables written for output_sample/ping_nemo.xml


In [9]:
#! head -n 5 output_sample/ping_nemo.xml

In [10]:
single_realms=[ ['ocean'], ['seaIce'],['ocnBgchem'], [ 'atmos'], ['land'],['landIce'], ['atmosChem'],[ 'aerosol' ]]
#single_realms=[]
for rs in single_realms :
    print rs[0]
    pingFileForRealmsList(rs[0],rs,svars,prefix=settings['ping_variables_prefix'],
                         comments=" ",exact=False, dummy=True,dummy_with_shape=True,
                         filename='output_sample/ping_%s.xml'%rs[0])


ocean
179 variables written for output_sample/ping_ocean.xml
seaIce
 99 variables written for output_sample/ping_seaIce.xml
ocnBgchem
223 variables written for output_sample/ping_ocnBgchem.xml
atmos
370 variables written for output_sample/ping_atmos.xml
land
318 variables written for output_sample/ping_land.xml
landIce
318 variables written for output_sample/ping_landIce.xml
atmosChem
370 variables written for output_sample/ping_atmosChem.xml
aerosol
156 variables written for output_sample/ping_aerosol.xml
